<a href="https://colab.research.google.com/github/yebiny/GANexample/blob/master/05_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DCGAN

In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
 
# mnist data 불러옴
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


## Parameters

In [0]:
# epoch, batch size, noise 로 쓸 상수 변수 선언
total_epoch = 100
batch_size = 100
n_noise = 100
 
# 전역 변수로 쓸 변수 0으로 초기화
D_global_step = tf.Variable(0, trainable=False, name='D_global_step')
G_global_step = tf.Variable(0, trainable=False, name='G_global_step')
 
# 이미지를 담는 변수 X와 노이즈 변수 Z
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Z = tf.placeholder(tf.float32, [None, n_noise])

is_training = tf.placeholder(tf.bool)

## Leak ReLU
* ReLU의 경우 음수영역에서 0을 출력하지만 이 함수는 음수영역에서 약간의 기울기를 가집니다.


In [0]:
def leaky_relu(x, leak=0.2):
    return tf.maximum(x, x * leak)

## Generator model
* DCGAN의 가장 큰 특징은 convolution lyaer와 batch normalization이고 마지막 output은 thnh가 사용됩니다.

In [0]:
def generator(noise):
    with tf.variable_scope('generator'):
        output = tf.layers.dense(noise, 128*7*7)
        output = tf.reshape(output, [-1, 7, 7, 128])
        output = tf.nn.relu(tf.layers.batch_normalization(output, training=is_training))
        output = tf.layers.conv2d_transpose(output, 64, [5, 5], strides=(2, 2), padding='SAME')
        output = tf.nn.relu(tf.layers.batch_normalization(output, training=is_training))
        output = tf.layers.conv2d_transpose(output, 32, [5, 5], strides=(2, 2), padding='SAME')
        output = tf.nn.relu(tf.layers.batch_normalization(output, training=is_training))
        output = tf.layers.conv2d_transpose(output, 1, [5, 5], strides=(1, 1), padding='SAME')
        output = tf.tanh(output)
    return output

## Discriminator model
* leaky_relu와 convolution lyaer를 사용합니다.
* 마지막에서 flatten 해줍니다.

In [0]:
def discriminator(inputs, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        
        # scope.reuse_variables()이후에는 같은 scope내에서 같은 name을 갖는 변수를 새로 만들지 않고 재사용 한다.
        if reuse:
            scope.reuse_variables()
            
        output = tf.layers.conv2d(inputs, 32, [5, 5], strides=(
            2, 2), padding='SAME')
        output = leaky_relu(output)
        output = tf.layers.conv2d(output, 64, [5, 5], strides=(2, 2), padding='SAME')
        output = leaky_relu(tf.layers.batch_normalization(output, training=is_training))
        output = tf.layers.conv2d(output, 128, [5, 5], strides=(2, 2), padding='SAME')
        output = leaky_relu(tf.layers.batch_normalization(output, training=is_training))
        flat = tf.contrib.layers.flatten(output)
        output = tf.layers.dense(flat, 1, activation=None)
    return output

## Noise

In [0]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1.0, 1.0, size=[batch_size, n_noise])
  
# interpolation 되는 과정을 보기위해 노이즈를 두개 생성하는 부분
def get_moving_noise(batch_size, n_noise):
    assert batch_size > 0
 
    # 노이즈 두개 생성
    noise_list = []
    base_noise = np.random.uniform(-1.0, 1.0, size=[n_noise])
    end_noise = np.random.uniform(-1.0, 1.0, size=[n_noise])
 
    # 임의의 step 설정
    step = (end_noise - base_noise) / batch_size
    noise = np.copy(base_noise)
    for _ in range(batch_size - 1):
        noise_list.append(noise)
        noise = noise + step
    noise_list.append(end_noise)   # 변해가는 모습을 볼 수 있게 base 부터 end 까지 노이즈 리스트 만듦
    
    return noise_list  

## Loss: Cross Entropy
* Cross entropy는주로 범주형 데이터에 사용되는 손실함ㅅ 입니다. 
* Cross entropy는 두 분포 사이의 유사성을 측정하는 척도입니다. 딥러닝에서의 분류 모델은 각 클래스의 확률값을 계산하므로 실제 클래스(y_true)와 모델이 예측한 클래스(y_pred)를 비교할 수 있으며 두 분포가 가까울수록 이 값은 더 작아집니다. 

In [11]:
G = generator(Z)
D_real = discriminator(X)
D_gene = discriminator(G, True)

# loss 함수 cross entropy 사용
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.zeros_like(D_gene)))
loss_D = loss_D_real + loss_D_gene
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

W0905 05:40:48.520984 139814144010112 deprecation.py:323] From <ipython-input-8-b822b833a4b3>:3: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0905 05:40:48.533602 139814144010112 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0905 05:40:48.829274 139814144010112 deprecation.py:323] From <ipython-input-8-b822b833a4b3>:5: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDAT

In [0]:
# tensorflow의 namefield를 이용해 변수를 재활용하기 위해 해당 collection의 variable들을 불러온다. 
# tf.get_collection(key)가 실행되면, key의 collection에 속하는 variable들의 리스트가 리턴된다.
# 출처: https://eyeofneedle.tistory.com/24 [Technology worth spreading]

vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='generator')
 
# training 시점에서 변수 update_ops에 업데이트가 저장됨. 그래서 update_ops를 학습할 때 넣어준다.
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [13]:
# loss를 adamoptimizer를 이용해서 minimize하게 학습
with tf.control_dependencies(update_ops):
    train_D = tf.train.AdamOptimizer().minimize(loss_D,var_list=vars_D, global_step=D_global_step)
    train_G = tf.train.AdamOptimizer().minimize(loss_G,var_list=vars_G, global_step=G_global_step)
    
# TensorBoard 사용하게 위해 loss값들을 기록
tf.summary.scalar('loss_D', loss_D)
tf.summary.scalar('loss_G', loss_G)    

<tf.Tensor 'loss_G:0' shape=() dtype=string>

## Model training

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
 
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs', sess.graph)  # 로그로 저장
 
    total_batch = int(mnist.train.num_examples / batch_size)  # 전체 배치 사이즈 계산
 
    for epoch in range(total_epoch):
        loss_val_D, loss_val_G = 0, 0
 
        batch_xs, batch_ys = None, None
        noise = None
 
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(-1, 28, 28, 1)
            noise = get_noise(batch_size, n_noise)
 
            # 학습 돌림 
            _, loss_val_D = sess.run([train_D, loss_D],feed_dict={X: batch_xs, Z: noise, is_training: True})
            _, loss_val_G = sess.run([train_G, loss_G],feed_dict={X: batch_xs, Z: noise, is_training: True})
 
        summary = sess.run(merged,feed_dict={X: batch_xs, Z: noise, is_training: True})
        writer.add_summary(summary, global_step=sess.run(G_global_step)) # log로 저장
 
        # epoch과 loss 프린트
        print('Epoch:', '%04d' % epoch,
            'D loss: {:.4}'.format(loss_val_D),
            'G loss: {:.4}'.format(loss_val_G))
     
        # 5번째 마다 그림 보여줌 및 저장
        if epoch == 0 or (epoch + 1) % 5 == 0:
            sample_size = 10
            noise = get_noise(sample_size, n_noise)
            samples = sess.run(G, feed_dict={Z: noise, is_training: False})
            test_noise = get_moving_noise(sample_size, n_noise)  # 두개의 노이즈 만들고 그 사이에 step을 더해서 noise list 가져옴
            test_samples = sess.run(G, feed_dict={Z: test_noise, is_training: False})
 
            fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))
 
            for i in range(sample_size):
                ax[0][i].set_axis_off()
                ax[1][i].set_axis_off()
                ax[0][i].imshow(np.reshape(samples[i], (28, 28)))  # generator가 만든 그림들
                ax[1][i].imshow(np.reshape(test_samples[i], (28, 28)))  # 두개의 노이즈를 만들어 interpolation되어가는 과정을 보여준 그림
             
            
            plt.savefig('{}.png'.format(str(epoch).zfill(3)),bbox_inches='tight')
            
            plt.close(fig)

Epoch: 0000 D loss: 0.1153 G loss: 9.229
Epoch: 0001 D loss: 0.0597 G loss: 4.931
Epoch: 0002 D loss: 0.08796 G loss: 6.33
Epoch: 0003 D loss: 0.04069 G loss: 5.273
Epoch: 0004 D loss: 0.06501 G loss: 5.908
Epoch: 0005 D loss: 0.3673 G loss: 3.824
Epoch: 0006 D loss: 0.4463 G loss: 3.121
Epoch: 0007 D loss: 0.3865 G loss: 3.38
Epoch: 0008 D loss: 0.4037 G loss: 3.401
Epoch: 0009 D loss: 0.1977 G loss: 3.38
Epoch: 0010 D loss: 0.9338 G loss: 2.324
Epoch: 0011 D loss: 0.6475 G loss: 2.663
Epoch: 0012 D loss: 1.285 G loss: 2.18
Epoch: 0013 D loss: 0.5323 G loss: 3.138
Epoch: 0014 D loss: 0.4924 G loss: 2.454
Epoch: 0015 D loss: 0.8241 G loss: 3.514
Epoch: 0016 D loss: 0.5668 G loss: 2.194
Epoch: 0017 D loss: 0.8821 G loss: 2.361
Epoch: 0018 D loss: 0.5723 G loss: 2.578
Epoch: 0019 D loss: 0.4671 G loss: 3.588
Epoch: 0020 D loss: 0.2647 G loss: 3.455
Epoch: 0021 D loss: 0.3037 G loss: 1.852
Epoch: 0022 D loss: 0.2212 G loss: 3.732
Epoch: 0023 D loss: 0.8343 G loss: 2.539
Epoch: 0024 D loss